In [ ]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Cria bases com nomes da bancada ruralista a partir de cruzamento de lista do TSE com deputados eleitos com nomes FPA
#

In [ ]:
import pandas as pd

In [3]:
# Carrega lista de deputados eleitos em 2014 - gerada com dados oficiais do http://www.tse.jus.br/eleitor-e-eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais¶

In [4]:
candidatos = pd.read_csv("candidatos_nome_urna_csv.csv",sep=';',encoding = 'latin_1')

In [5]:
eleitos = candidatos[(candidatos['Situacao'] == 'ELEITO POR QP') | (candidatos['Situacao'] == 'ELEITO POR MÉDIA') | (candidatos['Situacao'] == 'ELEITO')]

In [6]:
eleitos_d = eleitos[(eleitos['Cargo'] == 'DEPUTADO FEDERAL')]

In [7]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 23 to 26219
Data columns (total 7 columns):
Estado            513 non-null object
Cargo             513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null int64
Nome_urna         513 non-null object
Partido           513 non-null object
Situacao          513 non-null object
dtypes: int64(1), object(6)
memory usage: 32.1+ KB


# Carrega nomes da Frente parlamentar Agropecuária

In [8]:
# http://www.fpagropecuaria.org.br/integrantes

In [10]:
bancada = pd.read_csv("bancada_rur.csv",sep=';',encoding = 'latin_1')

In [11]:
bancada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 5 columns):
Nome                      257 non-null object
Nome_consulta_bancada     257 non-null object
Estado                    257 non-null object
Cargo_bancada_novembro    20 non-null object
Partido_novembro          257 non-null object
dtypes: object(5)
memory usage: 10.1+ KB


# Checa bancada ruralista com nomes de deputados eleitos

In [12]:
# Comparação pelo campo Nome urna da base do TSE, que deveria corresponder com o nome no site da Câmara

In [13]:
# É a comparação possível porque a base da bancada não tem o CPF

In [14]:
bancada_eleitos = pd.merge(eleitos_d, bancada, left_on='Nome_urna', right_on='Nome_consulta_bancada')

In [15]:
bancada_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 206
Data columns (total 12 columns):
Estado_x                  207 non-null object
Cargo                     207 non-null object
Nome_candidato            207 non-null object
CPF                       207 non-null int64
Nome_urna                 207 non-null object
Partido                   207 non-null object
Situacao                  207 non-null object
Nome                      207 non-null object
Nome_consulta_bancada     207 non-null object
Estado_y                  207 non-null object
Cargo_bancada_novembro    12 non-null object
Partido_novembro          207 non-null object
dtypes: int64(1), object(11)
memory usage: 21.0+ KB


In [16]:
# O cruzamento mostrou 50 nomes faltando - podem ser nomes com nome urna diferente ou suplentes

In [17]:
# Depois descobri que muitos deputados simplesmente trocam o nome como são chamados depois de eleitos

In [18]:
# Exemplo: LUIZ CARLOS RAMOS DO CHAPEU, que hoje só usa o nome LUIZ CARLOS RAMOS (http://www.camara.leg.br/internet/deputado/Dep_Detalhe.asp?id=5830522)

In [19]:
# Verifica nomes não encontrados

In [20]:
# outer: usa união de chaves de ambos os quadros, semelhante a uma junção externa completa do SQL; classificar chaves lexicograficamente
# indicator: Se True, adiciona uma coluna para saída DataFrame chamado "_merge" com informações sobre a origem de cada linha. Se uma seqüência de caracteres, a coluna com informações sobre a fonte de cada linha será adicionada para saída DataFrame, e a coluna será denominada valor da string. A coluna de informação é de tipo Categorical e assume um valor de "left_only" para observações cuja chave de mesclagem só aparece no DataFrame 'esquerdo', "right_only" para observações cuja chave de mesclagem aparece apenas no DataFrame 'direito' e "ambos" se o A chave de mesclagem da observação é encontrada em ambos.

In [22]:
naoencontrados = pd.merge(eleitos_d, 
                        bancada, 
                        left_on='Nome_urna', 
                        right_on='Nome_consulta_bancada',
                        how='outer',
                        indicator=True)

In [23]:
# Agrupa apenas pelo lado direito - a lista da bancada

In [24]:
rdf = naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [25]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 524 to 573
Data columns (total 12 columns):
Estado_x                  0 non-null object
Cargo                     0 non-null object
Nome_candidato            0 non-null object
CPF                       0 non-null float64
Nome_urna                 0 non-null object
Partido                   0 non-null object
Situacao                  0 non-null object
Nome                      50 non-null object
Nome_consulta_bancada     50 non-null object
Estado_y                  50 non-null object
Cargo_bancada_novembro    8 non-null object
Partido_novembro          50 non-null object
dtypes: float64(1), object(11)
memory usage: 5.1+ KB


In [24]:
rdf.to_csv('naoencontrados_bancada_ruralista.csv')

In [25]:
# Nesse ponto fiz uma checagem manual dos 50 nomes, verificando quem não era deputado eleito e atualizando os nomes urna

In [26]:
# Depois uni os dois arquivos em um

In [27]:
# Repeti o cruzamento e já inclui o CPF para essa base ser cruzada facilmente com outras

In [26]:
bancada_ruralista = pd.read_csv("dia22_11_bancadaruralista.csv",sep=',',encoding = 'utf_8')

In [27]:
bancada_ruralista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 4 columns):
Nome_candidato            196 non-null object
CPF                       196 non-null int64
Nome_urna                 196 non-null object
Cargo_bancada_novembro    12 non-null object
dtypes: int64(1), object(3)
memory usage: 6.2+ KB
